In [ ]:
!pip install python_binance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 3.5 MB/s 
     |████████████████████████████████| 106 kB 50.7 MB/s 
     |████████████████████████████████| 290 kB 55.0 MB/s 
     |████████████████████████████████| 749 kB 46.0 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


In [ ]:

import numpy as np
from binance.client import Client
from binance.enums import HistoricalKlinesType
import pandas as pd
import math
import random

class Get_data:
  def __init__(self  , coin = "XRPUSDT", start = "1 Jan, 2000", end = "12 May, 2029"):
    api_key = "dAtkqt2t1yonrWoyFMhZfh9Ln8lrAn0wvjPOUPu03ONEYRRioB4sOPlF8tjpp89l"
    secret_key = "BVoA041aA6Ca9nbUmbMub9U06JbGdZVu5mgIGjrVp3txoN7cVxcD1YKaTsd5Fhuu"
    client = Client(api_key, secret_key, requests_params = {"timeout":100 } )
    self.client = client
    self.interval = Client.KLINE_INTERVAL_30MINUTE
    self.coin = coin
    self.start = start
    self.end = end
    #self.df = get_data(clint,interval, coin , start , end )
  def get_data(self):
      columns = [ "Open time", "open","high", " low"  ,"close", "volume" , 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
      #columns = ["Open time", "open","high", " low"  ,"close", "volume"]
      candlesticks = self.client.get_historical_klines(self.coin, self.interval,self.start , self.end, klines_type  = HistoricalKlinesType.SPOT  )
      df = pd.DataFrame(candlesticks, columns = columns)
      #columns = [ "Open time", "open","high", " low"  ,"close", "volume" , 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume']
      #df = df[columns]
      df["Open time"] = pd.to_datetime(df["Open time"] ,unit='ms')
      return df

def create_labels( df, col_name = 'close', open_clm = 'open', window_size=11): 
  #Label code : BUY => 1, SELL => 0, HOLD => 2 for short and long
    """
    Data is labeled as per the logic in research paper
    Label code : BUY => 1, SELL => 0, HOLD => 2
    params :
        df => Dataframe with data
        col_name => name of column which should be used to determine strategy
    returns : numpy array with integer codes for labels with
              size = total-(window_size)+1
    """

    #self.log("creating label with original paper strategy")
    row_counter = 0
    total_rows = len(df)
    labels = np.zeros(total_rows)
    labels[:] = np.nan
    #print("Calculating labels...")
    #pbar = tqdm(total=total_rows)

    while row_counter < total_rows:
        if row_counter >= window_size - 1:
            window_begin = row_counter - (window_size - 1)
            window_end = row_counter
            window_middle = (window_begin + window_end) / 2

            min_ = np.inf
            min_index = -1
            max_ = -np.inf
            max_index = -1
            for i in range(window_begin, window_end + 1):
                price = float(df.iloc[i][col_name])
                open_price = float(df.iloc[i][open_clm])
                if open_price < min_:
                    min_ = open_price
                    min_index = i
                if price > max_:
                    max_ = price
                    max_index = i
            window_middle = int(window_middle)
            #print(window_middle)
            if max_index == window_middle:
                labels[window_middle] = 0
            elif min_index == window_middle:
                labels[window_middle] = 1
            else:
                labels[window_middle] = 2
        row_counter = row_counter + 1
        #pbar.update(1)
    #pbar.close()
    labels = [2.0 if np.isnan(x) else x for x in labels]
    return labels
    



def get_score(df, SL, output = 0):
    data = df
    #df['low'] = df[' low']
    time = df['open']
    holding = 0
    funds = initial_amount =10000
    #neat.nn.FeedForwardNetwork.create(genome, config)
    tax = 0
    Transaction = 0

    already_buyed = False
    already_selled = False
    stop_loss_trigger = False
    buying_probability = 0

    #varibles for going short
    number_of_share_use_for_short = 0
    buying_price = 0
    total_amount_for_short = 0
    profit_from_short = 0
    already_buyed = False
    already_selled = True 
    buyed_at = 0
    sl_count = 0
    total_trades = 0
    out_put=''

    f = open("Test_results.txt", "a")
    x_train=data
    for i in range(1, len(data)):
        if i == 4000:
          break
        label = data['Action'].values[i]
        if label == 1 and already_buyed == False:
            #print("buying", bp)
            if  True:
                out_put = " Signal = Opening Long ,\n Total coins  before Opening Long "+ str(holding) + " and total amount to invest = " + str(funds) + " \n"
                holding += float(funds*(1 - tax))/float(df['open'].values[i]) #Total amoutn invested - text  then divide
                funds = 0
                out_put = out_put + "Time: "+ str(time[i])+"  Open price: "+ str(df.open[i]) + "\n"
                #out_put =out_put + "Total coins after Opening LOng " +  str(holding) + " and total amount to invest = " + str(funds) + "\n"
                open_at = df['open'].values[i]
                already_buyed =True
                already_selled = False
                buyed_at = float(df['open'].values[i])
                Transaction =Transaction + 1
                total_trades += 1

                #print("Date: ", time[i] ,"  Open Price :  ",df.open[i], "Action Buying ..")
                out_put =out_put + "Date: " + str(time[i]) +"  Open Price :  " + str(df['open'].values[i]) +"Action Buying .." 
                out_put =out_put + " \n*******************************************"
                #print("Buying ...")
                #print('****************************************')
                if output:
                  print(out_put)
                #f.write(out_put)
        if label == 0 and already_selled == False:
            out_put =out_put + "\n Signal = Closing Long  \n Total coins  before Closing Long "+ str(holding) + " and total amount to invest = " + str(funds) + " \n"
            #print('selling', sp)
            if  True:
                funds = float(holding) * float(df['close'].values[i]) #* float(1-float(tax))
                holding = 0
                hasSold = True
                already_selled = True 
                already_buyed =False
                out_put =out_put + "Total coins after Closing LOng " +  str(holding) + " and total amount to invest = " + str(funds) + "\n"
                Transaction  =Transaction+ 1
                #print("Selling ...")

                out_put =out_put + "Date: "+ str(time[i] ) +  "  Closing Price :  "+ str(df.close[i]) + "Action Selling .."
                out_put =out_put + " \n*******************************************"
                if output:
                  print(out_put)
                #print("Date: ", time[i] , "  Closing Price :  ",df.close[i], "Action Selling ..")
                #print('****************************************')
                #f.write(out_put)
                #f.write(out_put)
    try:
      score = funds + holding*df['close'].values[i]
    except Exception as e:
      print("Error",e)
      score = 0
    #print(df.close[i])
    #print("SL hit = ", sl_count)
    if output:
      print("Total trades = ", total_trades)
    #detail = str(f" Funds: {funds}, Hold ETH: {holding}, Current score: {score}, Current Score in ETH:{score/df.close[i]}")#, "Signal = ", signal)
    return score
#get_score(df1, 0, output = 0)

In [ ]:
class Individual(object):
  
  '''
  Class representing individual in population
  '''
  def __init__(self, chromosome):
    self.chromosome = chromosome
    self.fitness =int(self.find_fitness())
    self.ETH = (self.fitness * 1.2)/2880
  
  def find_fitness(self):
    df_orignal["Action"] = self.chromosome
    score  =get_score(df_orignal, 0, output = 0)
    return score



# objective function
def objective_label( best_label_interval,  pervious_value, value ):
  data = df1[pervious_value: value]
  data['Action'] = best_label_interval
  data = data.reset_index()
  scr = get_score(data, 0, output = 0)
  return scr



def get_interval_labels(pop, interval = 1800):
  interval_list =list(range(interval,len(pop[0]), interval))
  interval_list.append(len(pop[0]))
  pervious_value = 0
  final_label = []
  for value in interval_list:
    best_label_interval = pop[5][pervious_value: value]
    score = objective_label(best_label_interval, pervious_value, value )
    #print(value, " score: ", score)
    for p in pop:
      label_interval = p[pervious_value: value]
      label_score = objective_label(label_interval,  pervious_value, value )
      #print("Label score", label_score)
      if label_score > score:
        best_label_interval =label_interval
        score = label_score
        #print("Best_Score = ", label_score)
      #print(p[pervious_value: value])
    #print("len of final label ", len(final_label))
    final_label = final_label + best_label_interval
    pervious_value = value
  return final_label

def initialPopulatation(size):
  print("Initialing initial populatation ...")
  populatation = []
  for i in range(1,size+1):
    labels = create_labels(df1 , window_size=i )#Label code : BUY => 1, SELL => 0, HOLD => 2
    #print("labels length ",len(labels))
    if len(set(labels)) != 3:
      labels[i] = 0.0
      labels[i+1] = 1.0
      labels[i+2] = 2.0
    df_orignal['Action'] = labels
    #trade(df_orignal,10000,'k')
    #X_train,Y_train,X_val,y_val = prepare_data_for_Model(df_orignal,test_size= 0.00001,oversmpl= True)
    #model = create_train_Model(X_train,Y_train,X_val,y_val )

    populatation.append(Individual(labels))
  print("Completed")
  return populatation

def selection(populatation):
  selected= []
  pop_length = len(populatation)
  pop = [x.chromosome for x in  populatation]
  for i in range(pop_length):
    final_label = get_interval_labels(pop, interval =interval_list.pop(random.randrange(len(interval_list))))
    populatation.append(Individual(final_label))

  populatation = sorted(populatation, key = lambda x:x.fitness,reverse= True)
  # if len(populatation) % 2 == 0:
  #   half= int(len(populatation)/2)
  # else:
  #   half= int(math.ceil(len(populatation)/2))
  return populatation[:pop_length]


def crossover(pop,pop_size):
  size = len(pop)
  for i in range(0,size-1):# Loop for each chromosome for crossover with the next chromosome
    prnt1 = pop[i]
    prnt2 = pop[i+1]

    if i == size-2:   #Final chromosome will crossover with the first one
      prnt1 = pop[i+1]
      prnt2 = pop[0]
      #print(prnt1,prnt2)
    p =prnt1.chromosome# selected_chromosome[0].chromosome
    p1 =prnt2.chromosome#selected_chromosome[1].chromosome
    half = int(len(p)/2)
    newA =p[:half]#,p[half:]     # Half of the first chromosome
    newB = p1[:half]         # Half of the secound chromosome
    for i in p1[half:]:     
      newA.append(i)  # Adding the secound half of the secound chromosome.
    for i in p[half:]:
      newB.append(i)
    if len(pop) != pop_size:
      pop.append(Individual(newA))
    if len(pop) != pop_size:
      pop.append(Individual(newB))
  return pop



def mutation(pop, percentage_mutation = 50,number_bit_mutate=5):
  for i in range(0,len(pop)):
    rndm_no = random.randint(0,100)
    if rndm_no < 10 and i !=0:
      for x in range(len(pop[i].chromosome)):
        pop[i].chromosome[x] =  float(random.randint(0,2))
      pop[i].find_fitness()
    elif rndm_no < percentage_mutation and i !=0:
      # flip the bit
      for j in range(0,number_bit_mutate):
        rn = random.randint(0,len(pop[i].chromosome)-1) # random number
        pop[i].chromosome[rn] =  random.randint(0,2)
      #print("before",pop[i].fitness)
      pop[i].find_fitness()
      #print("after",pop[i].fitness)
  return pop
#mutation(new_populatation,100)

def save_data():
  labels = populatation[0].chromosome     #[x.fitness for x in populatation]#,[x.ETH for x in populatation]
  #print(len(labels))
  #df_orignal = get_data().head(2000)
  print(len(df_orignal))
  df_orignal['Action'] = labels
  df_orignal.to_csv("Data_with_lebals1.csv")
#save_data()


# final_label = get_interval_labels(pop, interval =interval_list.pop(random.randrange(len(interval_list))))
# objective(final_label)

In [ ]:
# final_label = get_interval_labels(pop, interval =interval_list.pop(random.randrange(len(interval_list))))
# populatation.append(Individual(final_label))

In [ ]:
populatation_size  = 50
total_generations = 100

total_amout = 10000
df = Get_data(coin='BTCUSDT', start = "1 Sep, 2000",  end = "1 Jan, 2021").get_data()
time = df['Open time']
del df['Open time']
df = df.astype(float)
df_orignal = df1 =df
populatation = initialPopulatation(populatation_size)
fee = 0

interval_list = list(range(24,int(len(populatation[0].chromosome)/2),5))
for i in range(total_generations):
  print("\n\n Generation : ", i)
  populatation = sorted(populatation,key=lambda x:x.fitness, reverse= True)
  print('At start  populatation Reults, ',[x.fitness for x in populatation])
  #print(len(populatation))
  new_populatation = selection(populatation)
  #print(new_populatation)
  #new_populatation =selected_chromosome # crossover(selected_chromosome, len(populatation))
  #new_populatation = sorted(new_populatation, key = lambda x:x.fitness,reverse= True)
   
  print('After Cross over  populatation Reults, ',[x.fitness for x in new_populatation])

  new_populatation_mutated = mutation(new_populatation,percentage_mutation = 50,number_bit_mutate=10)
  print('After Mutation results , ',[x.fitness for x in new_populatation_mutated])
  populatation = new_populatation_mutated
  populatation = sorted(populatation, key = lambda x:x.fitness,reverse= True)
  print("length of lebals in the generation ", len(populatation[0].chromosome))
  if i%10 == 0:
    save_data()

[x.fitness for x in populatation],[x.ETH for x in populatation]



Initialing initial populatation ...
Completed


 Generation :  0
At start  populatation Reults,  [98071956, 74091751, 55476808, 44762668, 30119603, 28509555, 28134549, 16242047, 15249696, 8600536, 8333006, 5136385, 4917318, 2879370, 2730463, 2068215, 1942164, 1661868, 1637028, 1245984, 1116791, 1096648, 936386, 932697, 853384, 700352, 640685, 639619, 557182, 555376, 530395, 485453, 476888, 470383, 449367, 447086, 426055, 400369, 393707, 392057, 392057, 391639, 388152, 381415, 381415, 365468, 353794, 327906, 317680, 10005]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


After Cross over  populatation Reults,  [98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956]
After Mutation results ,  [98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98071956, 98

In [ ]:
len(final_label), len(pop[0])

In [ ]:
#(767689.6879827107, 767689.6879827107)
(9999.956941911194, 765956.194750725)